# Nuage Compta et la facture s'analysa

## 0. Import des librairies

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from keras.models import load_model

Using TensorFlow backend.


In [3]:
path = "assets\imagedentrainement.jpeg"
img_originale = cv2.imread(path)
model = pickle.load(open("mon_model.h5","rb"))

In [4]:
def evaluate(image_lettre):

    alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z', '<3']

    image_lettre = np.expand_dims(image_lettre, axis=(0,-1))
    prediction = np.argmax(model.predict(image_lettre) , axis=1)[0]
    lettre = alphabet[prediction]

    return lettre

In [5]:
def resize_image(img, size=(20,20)):

    h, w = img.shape[:2]
    
    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w


    if dif > (size[0] + size[1]):
        interpolation = cv2.INTER_AREA
    else:
        interpolation = cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    mask = np.zeros((dif, dif), dtype=img.dtype)
    mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]

    return cv2.resize(mask, size, interpolation)

In [6]:
img_traite = img_originale.copy()

img_traite = cv2.cvtColor(img_traite, cv2.COLOR_BGR2GRAY)
imgCanny = cv2.Canny(img_traite, 80, 80)
kernel = np.ones((5, 5))
imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
contours = cv2.findContours(imgDil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]

img_affichage = img_traite.copy()

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    image_lettre = img_affichage[y:y+h, x:x+w]

    image_lettre = cv2.bitwise_not(image_lettre)
    image_lettre = resize_image(image_lettre)
    image_lettre = cv2.copyMakeBorder(image_lettre, 4, 4, 4, 4, cv2.BORDER_CONSTANT)

    predictions = list()
    lettre = evaluate(image_lettre)

    cv2.putText(img_affichage, lettre, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
    cv2.rectangle(img_affichage, (x , y ), (x + w , y + h ), (0, 0, 0), 5)

print(img_originale.shape)
print(img_traite.shape)
print(img_affichage.shape)

cv2.imshow("image", img_affichage)
cv2.waitKey()
cv2.destroyWindow("image")


del(img_traite, imgCanny, image_lettre, img_affichage)

(592, 1504, 3)
(592, 1504)
(592, 1504)
